# DocQGenius: Document Insights Assistant

#### This system uses Langchain, Astra DB and OpenAI (ChatGPT) to analyze the given PDF dosument and Answer questions related to that specific document only. Extremely useful while researching particular topics from a reference book.



DataStax Astra DB is a cloud-native, scalable Database-as-a-Service built on Apache Cassandra. Apache Cassandra is the open-source NoSQL database behind the largest applications in the world, including Netflix and Instagram.

### Installing the required dependencies:

In [ ]:
!pip install -q cassio datasets langchain openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 860.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.9/266.9 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.2/279.2 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━


Langchain: Langchain is a software framework designed for natural language processing tasks, offering a suite of tools and libraries to streamline the development of language-based applications and solutions. We will be connecting the OpenAI API using Langchain.

OpenAI API: The OpenAI API provides access to cutting-edge artificial intelligence models developed by OpenAI, enabling developers to integrate powerful language understanding, generation, and other AI capabilities into their applications via a simple API interface.

CassIO: We will initialize the DB connection with CassIO, the engine powering the Astra DB integration in LangChain.

### Importing the required packages:

In [ ]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

from datasets import load_dataset

import cassio

In [ ]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.5 MB/s eta 0:00:00


PyPDF2 is a Python library for working with PDF files. It allows users to extract text, merge multiple PDFs, split PDFs into individual pages, rotate pages by providing a comprehensive toolkit for manipulating PDF documents programmatically.

In [ ]:
from PyPDF2 import PdfReader

### Setting up the token, DB ID and API key

In [ ]:
ASTRA_DB_APPLICATION_TOKEN = "secret"
ASTRA_DB_ID = "secret"

OPENAI_API_KEY = "secret"

### Getting the path of the PDF

In [ ]:
pdfreader = PdfReader('proposal.pdf')

In [ ]:
from typing_extensions import Concatenate

raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [ ]:
raw_text

"Java\nProject\nProposal\nFinFlow:\nThe\nexpense\ntracker\nGroup\nMembers:\nWilliam\nYe\n(\nwy2399@nyu.edu\n),\nSiddhant\nKulkarni\n(\nsk10841@nyu.edu\n),\nSwapnamoy\nBhowmick\n(\nsb9180@nyu.edu\n)\nApplication\nDescription:\nThe\nJa v a\nExpense\nT r ack er\nis\na\nuser-friendly\ndeskt op\napplication\ndesigned\nt o\nhelp\nindividuals\nmanage\ntheir\npersonal\nﬁnances\nwith\nease.\nBuilt\nusing\nJa v a,\nthis\napplication\nwill\npr o vide\na\nstr aightfor war d\ninter face\nfor\ntr acking\ndaily\nexpenses,\ncategorizing\nthem\nand\nr e viewing\nﬁnancial\nhabits\no v er\ntime.\nK e y\nF eatur es:\n●\nUser\nA uthentication:\nSecur e\nlogin\nand\nr egistr ation\nfunctionality .\n●\nT r ansaction\nLogging:\nUsers\ncan\nquickly\nlog\ne v er y\nexpense\nor\nincome,\nspecifying\nthe\namount,\ndate,\ncategor y\n(e.g.,\ngr oceries,\nutilities,\nsalar y)\nand\na\nsmall\nnote\nfor\nfutur e\nr ef er ence.\nUsers\ncan\nalso\nedit\nan\nexisting\nr ecor d\nor\ndelete\nit.\n●\nCategor y\nManagement:\

### Initializing the connection to the database

In [ ]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

ERROR:cassandra.connection:Closing connection <LibevConnection(138189540390224) c4c5a4ad-f352-4c82-8ac7-68a9f516a28a-us-east1.db.astra.datastax.com:29042:3c39acb1-b6e7-41d7-8480-76e8e4bf63c6> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


### Creating the LangChain embedding and LLM objects

In [ ]:
llm = OpenAI(openai_api_key=OPENAI_API_KEY)
embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


### Creating the LangChain vector store backed by Astra DB.

We will split the text using Character Text Split such that it should not increse the token size

In [ ]:
astra_vector_store = Cassandra(
    embedding=embedding,
    table_name="qa_mini_demo",
    session=None,
    keyspace=None,
)

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [ ]:
texts[:50]

['Java\nProject\nProposal\nFinFlow:\nThe\nexpense\ntracker\nGroup\nMembers:\nWilliam\nYe\n(\nwy2399@nyu.edu\n),\nSiddhant\nKulkarni\n(\nsk10841@nyu.edu\n),\nSwapnamoy\nBhowmick\n(\nsb9180@nyu.edu\n)\nApplication\nDescription:\nThe\nJa v a\nExpense\nT r ack er\nis\na\nuser-friendly\ndeskt op\napplication\ndesigned\nt o\nhelp\nindividuals\nmanage\ntheir\npersonal\nﬁnances\nwith\nease.\nBuilt\nusing\nJa v a,\nthis\napplication\nwill\npr o vide\na\nstr aightfor war d\ninter face\nfor\ntr acking\ndaily\nexpenses,\ncategorizing\nthem\nand\nr e viewing\nﬁnancial\nhabits\no v er\ntime.\nK e y\nF eatur es:\n●\nUser\nA uthentication:\nSecur e\nlogin\nand\nr egistr ation\nfunctionality .\n●\nT r ansaction\nLogging:\nUsers\ncan\nquickly\nlog\ne v er y\nexpense\nor\nincome,\nspecifying\nthe\namount,\ndate,\ncategor y\n(e.g.,\ngr oceries,\nutilities,\nsalar y)\nand\na\nsmall\nnote\nfor\nfutur e',
 'functionality .\n●\nT r ansaction\nLogging:\nUsers\ncan\nquickly\nlog\ne v er y\nexpense\nor\nincome,\

### Loading the dataset into the vector store



In [ ]:

astra_vector_store.add_texts(texts[:50])

print("Inserted %i headlines." % len(texts[:50]))

astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 6 headlines.


### Running the model

We can either type in the question we want to find out related to the PDF document or we can type "quit" to exit the program.

In [ ]:
first_question = True

import warnings
warnings.filterwarnings("ignore")

while True:
    if first_question:
        query_text = input("\nEnter your question (or type 'quit' to exit): ").strip()
    else:
        query_text = input("\nWhat's your next question (or type 'quit' to exit): ").strip()

    if query_text.lower() == "quit":
        break

    if query_text == "":
        continue

    first_question = False



    print("\nQUESTION: \"%s\"" % query_text)
    answer = astra_vector_index.query(query_text, llm=llm).strip()
    print("ANSWER: \"%s\"\n" % answer)


Enter your question (or type 'quit' to exit): What are the names of the group members?

QUESTION: "What are the names of the group members?"


ANSWER: "I don't know."


What's your next question (or type 'quit' to exit): What are the java features?

QUESTION: "What are the java features?"


ANSWER: "The Java features mentioned in the context are multithreading, Java Swing for GUI, Java SE features such as Collections and IO, and Spring Boot for RESTful API."


What's your next question (or type 'quit' to exit): group members?

QUESTION: "group members?"


ANSWER: "The application allows for splitting expenses among multiple people, but it does not mention anything about group members specifically."


What's your next question (or type 'quit' to exit): key features?

QUESTION: "key features?"


ANSWER: "Some key features mentioned in the context are multithreading, Java Swing for GUI, Java SE features, and Spring Boot for RESTful API."


What's your next question (or type 'quit' to exit): what is the application description and what are the features of this application?

QUESTION: "what is the application description and what are the features of this application?"


ANSWER: "The application is described as being capable of handling data access and manipulation efficiently, utilizing multithreading to process multiple user actions concurrently. It also utilizes Java Swing for building the graphical user interface and leverages core Java features such as Collections and IO. Additionally, it implements Spring Boot for RESTful API in the back end."


What's your next question (or type 'quit' to exit): is there python code in it?

QUESTION: "is there python code in it?"


ANSWER: "No, there is no mention of Python code in the given context."


What's your next question (or type 'quit' to exit): quit
